# Circuit VS_SW_RL1

In [ ]:
import math
import villas.dataprocessing.readtools as rt
import villas.dataprocessing.plottools as pt
from villas.dataprocessing.timeseries import TimeSeries as ts
import matplotlib.pyplot as plt
import numpy as np
import dpsimpy

#%matplotlib widget

## SP for Initialization

In [ ]:
sim_name_pf = 'SP_1Ph_VS_SW_RL1_Init'
dpsimpy.Logger.set_log_dir("logs/" + sim_name_pf)

# Nodes
gnd_pf = dpsimpy.sp.SimNode.gnd
n1_pf  = dpsimpy.sp.SimNode('n1_pf')
n2_pf  = dpsimpy.sp.SimNode('n2_pf')
n3_pf  = dpsimpy.sp.SimNode('n2_pf')

resistance = 5
inductance = 0.02
open_resistance = 1e6
closed_resistance = 0.001

# Components
vs_pf = dpsimpy.sp.ph1.VoltageSource('vs')
vs_pf.set_parameters(V_ref=complex(10,0), f_src=50) # Vref is phase-to-phase RMS
sw_pf = dpsimpy.sp.ph1.Resistor('r1_pf')
sw_pf.set_parameters(closed_resistance)
r1_pf = dpsimpy.sp.ph1.Resistor('r1_pf')
r1_pf.set_parameters(resistance)
l1_pf = dpsimpy.sp.ph1.Inductor('l1_pf')
l1_pf.set_parameters(inductance)


# Connections
vs_pf.connect([gnd_pf, n1_pf])
sw_pf.connect([n1_pf, n2_pf])
r1_pf.connect([n2_pf, n3_pf])
l1_pf.connect([gnd_pf, n3_pf])

# Define system topology
system_pf = dpsimpy.SystemTopology(50, [n1_pf, n2_pf, n3_pf], [vs_pf, sw_pf, r1_pf, l1_pf])

# Logging
logger_pf = dpsimpy.Logger(sim_name_pf)
logger_pf.log_attribute('n1.v', 'v', n1_pf)
logger_pf.log_attribute('n2.v', 'v', n2_pf)
logger_pf.log_attribute('n2.v', 'v', n3_pf)

sim_pf = dpsimpy.Simulation(sim_name_pf, dpsimpy.LogLevel.debug)
sim_pf.set_system(system_pf)
sim_pf.set_domain(dpsimpy.Domain.SP)
sim_pf.set_time_step(0.1)
sim_pf.set_final_time(0.5)
sim_pf.add_logger(logger_pf)
sim_pf.run()

## EMT 1Ph

In [ ]:
time_step = 0.00001
final_time = 2
sim_name = 'EMT_1Ph_VS_SW_RL1'
dpsimpy.Logger.set_log_dir('logs/' + sim_name)

gnd = dpsimpy.emt.SimNode.gnd
n1 = dpsimpy.emt.SimNode('n1')
n1.set_initial_voltage(sim_pf.get_idobj_attr(n1_pf.name(), 'v').get()[0][0])
n2 = dpsimpy.emt.SimNode('n2')
n2.set_initial_voltage(sim_pf.get_idobj_attr(n2_pf.name(), 'v').get()[0][0])
n3 = dpsimpy.emt.SimNode('n3')
n3.set_initial_voltage(sim_pf.get_idobj_attr(n3_pf.name(), 'v').get()[0][0])

resistance = 5
inductance = 0.02
open_resistance = 1e6
closed_resistance = 0.001

vs = dpsimpy.emt.ph1.VoltageSource('vs')
vs.set_parameters(complex(10, 0)* math.sqrt(2/3), 50) # still using deprecated class that is initialised with Vref phase-to-ground peak voltage
res = dpsimpy.emt.ph1.Resistor('Resistor')
res.set_parameters(resistance)
ind = dpsimpy.emt.ph1.Inductor('Inductance')
ind.set_parameters(inductance)
sw = dpsimpy.emt.ph1.Switch('Switch')
sw.set_parameters(open_resistance, closed_resistance, True)

vs.connect([gnd, n1])
sw.connect([n1, n2])
res.connect([n2, n3])
ind.connect([n3, gnd])

sys = dpsimpy.SystemTopology(50, [n1, n2, n3], [vs, sw, res, ind])

logger = dpsimpy.Logger(sim_name)
logger.log_attribute('n1.v', 'v', n1)
logger.log_attribute('n2.v', 'v', n2)
logger.log_attribute('n3.v', 'v', n3)
logger.log_attribute('r1.i_intf', 'i_intf', res)

sim = dpsimpy.Simulation(sim_name)
sim.set_domain(dpsimpy.Domain.EMT)
sim.set_solver(dpsimpy.Solver.MNA)
sim.set_system(sys)
sim.set_time_step(time_step)
sim.set_final_time(final_time)
sim.add_logger(logger)

event_sw1 = dpsimpy.event.SwitchEvent(0.5, sw, False)
event_sw2 = dpsimpy.event.SwitchEvent(1.5, sw, True)
sim.add_event(event_sw1)
sim.add_event(event_sw2)

sim.run()

## DP 1Ph

In [ ]:
time_step = 0.00001
final_time = 2
sim_name = 'DP_1Ph_VS_SW_RL1'
dpsimpy.Logger.set_log_dir("logs/" + sim_name)

# Nodes
gnd = dpsimpy.dp.SimNode.gnd
n1 =  dpsimpy.dp.SimNode('n1')
n1.set_initial_voltage(sim_pf.get_idobj_attr(n1_pf.name(), 'v').get()[0][0])
n2 =  dpsimpy.dp.SimNode('n2')
n2.set_initial_voltage(sim_pf.get_idobj_attr(n2_pf.name(), 'v').get()[0][0])
n3 =  dpsimpy.dp.SimNode('n3')
n3.set_initial_voltage(sim_pf.get_idobj_attr(n3_pf.name(), 'v').get()[0][0])

resistance = 5
inductance = 0.02
open_resistance = 1e6
closed_resistance = 0.001

# Components
vs = dpsimpy.dp.ph1.VoltageSource('vs')
vs.set_parameters(V_ref=complex(10,0))
res = dpsimpy.dp.ph1.Resistor('r1')
res.set_parameters(resistance)
ind = dpsimpy.dp.ph1.Inductor('l1')
ind.set_parameters(inductance)
sw = dpsimpy.dp.ph1.Switch('Switch')
sw.set_parameters(open_resistance, closed_resistance, True)

vs.connect([gnd, n1])
sw.connect([n1, n2])
res.connect([n2, n3])
ind.connect([n3, gnd])

sys = dpsimpy.SystemTopology(50, [gnd, n1, n2, n3], [vs, sw, ind, res])

logger = dpsimpy.Logger(sim_name)
logger.log_attribute('n1.v', 'v', n1);
logger.log_attribute('n2.v', 'v', n2);
logger.log_attribute('n3.v', 'v', n3);
logger.log_attribute('r1.i_intf', 'i_intf', res);

sim = dpsimpy.Simulation(sim_name)
sim.set_domain(dpsimpy.Domain.DP)
sim.set_solver(dpsimpy.Solver.MNA)
sim.set_system(sys)
sim.set_time_step(time_step)
sim.set_final_time(final_time)
sim.add_logger(logger)

event_sw1 = dpsimpy.event.SwitchEvent(0.5, sw, False)
event_sw2 = dpsimpy.event.SwitchEvent(1.5, sw, True)
sim.add_event(event_sw1)
sim.add_event(event_sw2)

sim.run()

## SP 1Ph

In [ ]:
time_step = 0.00001
final_time = 2
sim_name = 'SP_1Ph_VS_SW_RL1'
dpsimpy.Logger.set_log_dir("logs/" + sim_name)

# Nodes
gnd = dpsimpy.sp.SimNode.gnd
n1 =  dpsimpy.sp.SimNode('n1')
n1.set_initial_voltage(sim_pf.get_idobj_attr(n1_pf.name(), 'v').get()[0][0])
n2 =  dpsimpy.sp.SimNode('n2')
n2.set_initial_voltage(sim_pf.get_idobj_attr(n2_pf.name(), 'v').get()[0][0])
n3 =  dpsimpy.sp.SimNode('n3')
n3.set_initial_voltage(sim_pf.get_idobj_attr(n3_pf.name(), 'v').get()[0][0])

resistance = 5
inductance = 0.02
open_resistance = 1e6
closed_resistance = 0.001

# Components
vs = dpsimpy.sp.ph1.VoltageSource('vs')
vs.set_parameters(V_ref=complex(10,0))
res = dpsimpy.sp.ph1.Resistor('r1')
res.set_parameters(resistance)
ind = dpsimpy.sp.ph1.Inductor('l1')
ind.set_parameters(inductance)
sw = dpsimpy.sp.ph1.Switch('Switch')
sw.set_parameters(open_resistance, closed_resistance, True)

vs.connect([gnd, n1])
sw.connect([n1, n2])
res.connect([n2, n3])
ind.connect([n3, gnd])

sys = dpsimpy.SystemTopology(50, [gnd, n1, n2, n3], [vs, sw, ind, res])

logger = dpsimpy.Logger(sim_name)
logger.log_attribute('n1.v', 'v', n1);
logger.log_attribute('n2.v', 'v', n2);
logger.log_attribute('n3.v', 'v', n3);
logger.log_attribute('r1.i_intf', 'i_intf', res);

sim = dpsimpy.Simulation(sim_name)
sim.set_domain(dpsimpy.Domain.SP)
sim.set_solver(dpsimpy.Solver.MNA)
sim.set_system(sys)
sim.set_time_step(time_step)
sim.set_final_time(final_time)
sim.add_logger(logger)

event_sw1 = dpsimpy.event.SwitchEvent(0.5, sw, False)
event_sw2 = dpsimpy.event.SwitchEvent(1.5, sw, True)
sim.add_event(event_sw1)
sim.add_event(event_sw2)

sim.run()

## EMT 3Ph

In [ ]:
time_step = 0.00001
final_time = 2
sim_name = 'EMT_3Ph_VS_SW_RL1'
dpsimpy.Logger.set_log_dir('logs/' + sim_name)

gnd = dpsimpy.emt.SimNode.gnd
n1 = dpsimpy.emt.SimNode('n1', dpsimpy.PhaseType.ABC)
n1.set_initial_voltage(sim_pf.get_idobj_attr(n1_pf.name(), 'v').get()[0][0])
n2 = dpsimpy.emt.SimNode('n2', dpsimpy.PhaseType.ABC)
n2.set_initial_voltage(sim_pf.get_idobj_attr(n2_pf.name(), 'v').get()[0][0])
n3 = dpsimpy.emt.SimNode('n3', dpsimpy.PhaseType.ABC)
n3.set_initial_voltage(sim_pf.get_idobj_attr(n3_pf.name(), 'v').get()[0][0])

resistance = 5
inductance = 0.02
open_resistance = 1e6
closed_resistance = 0.001

vs = dpsimpy.emt.ph3.VoltageSource('vs')
vs.set_parameters(V_ref=dpsimpy.Math.single_phase_variable_to_three_phase(complex(10, 0)), f_src=50)
res = dpsimpy.emt.ph3.Resistor('Resistor')
res.set_parameters(dpsimpy.Math.single_phase_parameter_to_three_phase(resistance))
ind = dpsimpy.emt.ph3.Inductor('Inductance')
ind.set_parameters(dpsimpy.Math.single_phase_parameter_to_three_phase(inductance))
sw = dpsimpy.emt.ph3.Switch('Switch')
sw.set_parameters(dpsimpy.Math.single_phase_parameter_to_three_phase(open_resistance), dpsimpy.Math.single_phase_parameter_to_three_phase(closed_resistance), True)
sw.close()

vs.connect([gnd, n1])
sw.connect([n1, n2])
res.connect([n2, n3])
ind.connect([n3, gnd])

sys = dpsimpy.SystemTopology(50, [gnd, n1, n2, n3], [vs, sw, res, ind])

logger = dpsimpy.Logger(sim_name)
logger.log_attribute('n1.v', 'v', n1);
logger.log_attribute('n2.v', 'v', n2);
logger.log_attribute('n3.v', 'v', n3);
logger.log_attribute('r1.i_intf', 'i_intf', res)

sim = dpsimpy.Simulation(sim_name)
sim.set_domain(dpsimpy.Domain.EMT)
sim.set_solver(dpsimpy.Solver.MNA)
sim.set_system(sys)
sim.set_time_step(time_step)
sim.set_final_time(final_time)
sim.add_logger(logger)

event_sw1 = dpsimpy.event.SwitchEvent3Ph(0.5, sw, False)
event_sw2 = dpsimpy.event.SwitchEvent3Ph(1.5, sw, True)
sim.add_event(event_sw1)
sim.add_event(event_sw2)

sim.run()

# Read results

## EMT 3Ph results

In [ ]:
# read 
work_dir = 'logs/EMT_3Ph_VS_SW_RL1/'
log_name = 'EMT_3Ph_VS_SW_RL1'
print(work_dir + log_name + '.csv')

ts_dpsim_emt_3ph = rt.read_timeseries_dpsim(work_dir + log_name + '.csv')

## EMT 1Ph results

In [ ]:
work_dir = 'logs/EMT_1Ph_VS_SW_RL1/'
log_name = 'EMT_1Ph_VS_SW_RL1'
print(work_dir + log_name + '.csv')

ts_dpsim_emt_1ph = rt.read_timeseries_dpsim(work_dir + log_name + '.csv')

## DP 1Ph results

In [ ]:
work_dir = 'logs/DP_1Ph_VS_SW_RL1/'
log_name = 'DP_1Ph_VS_SW_RL1'
print(work_dir + log_name + '.csv')

ts_dpsim_dp_1ph = rt.read_timeseries_dpsim(work_dir + log_name + '.csv')

# shifted
ts_dpsim_dp_1ph_shift = ts.frequency_shift_list(ts_dpsim_dp_1ph, 50)

# phase-to-phase RMS to phase-to-ground Peak
for name, ts in ts_dpsim_dp_1ph_shift.items():
    ts.values = ts.values * math.sqrt(2/3)

## SP 1Ph results

In [ ]:
work_dir = 'logs/SP_1Ph_VS_SW_RL1/'
log_name = 'SP_1Ph_VS_SW_RL1'
print(work_dir + log_name + '.csv')

ts_dpsim_sp_1ph = rt.read_timeseries_dpsim(work_dir + log_name + '.csv')

# shifted
ts_dpsim_sp_1ph_shift = ts.frequency_shift_list(ts_dpsim_sp_1ph, 50)

# phase-to-phase RMS to phase-to-ground Peak
for name, ts in ts_dpsim_sp_1ph_shift.items():
    ts.values = ts.values * math.sqrt(2/3)

# Plot results

In [ ]:
plt.close('all')

# EMT 3Ph
ts_dpsim_emt_3ph['n1.v_0'].label = 'v1 EMT 3Ph'
ts_dpsim_emt_3ph['n2.v_0'].label = 'v2 EMT 3Ph'
ts_dpsim_emt_3ph['n3.v_0'].label = 'v3 EMT 3Ph'
ts_dpsim_emt_3ph['r1.i_intf_0'].label = 'i12 EMT 3Ph'
pt.plot_timeseries(1, ts_dpsim_emt_3ph['n1.v_0'], plt_linestyle=':')
pt.plot_timeseries(2, ts_dpsim_emt_3ph['n2.v_0'], plt_linestyle=':')
pt.plot_timeseries(3, ts_dpsim_emt_3ph['n3.v_0'], plt_linestyle=':')
pt.plot_timeseries(4, ts_dpsim_emt_3ph['r1.i_intf_0'], plt_linestyle=':')

# EMT 1Ph
ts_dpsim_emt_1ph['n1.v'].label = 'v1 EMT 1Ph'
ts_dpsim_emt_1ph['n2.v'].label = 'v2 EMT 1Ph'
ts_dpsim_emt_1ph['n3.v'].label = 'v3 EMT 1Ph'
ts_dpsim_emt_1ph['r1.i_intf'].label = 'i12 EMT 1Ph'
pt.plot_timeseries(1, ts_dpsim_emt_1ph['n1.v'], plt_linestyle='-')
pt.plot_timeseries(2, ts_dpsim_emt_1ph['n2.v'], plt_linestyle='-')
pt.plot_timeseries(3, ts_dpsim_emt_1ph['n3.v'], plt_linestyle='-')
pt.plot_timeseries(4, ts_dpsim_emt_1ph['r1.i_intf'], plt_linestyle='-')

# DP 1Ph shifted
ts_dpsim_dp_1ph_shift['n1.v_shift'].label = 'v1 DP 1Ph'
ts_dpsim_dp_1ph_shift['n2.v_shift'].label = 'v2 DP 1Ph'
ts_dpsim_dp_1ph_shift['n3.v_shift'].label = 'v3 DP 1Ph'
ts_dpsim_dp_1ph_shift['r1.i_intf_shift'].label = 'i12 DP 1Ph'
pt.plot_timeseries(1, ts_dpsim_dp_1ph_shift['n1.v_shift'], plt_linestyle='-')
pt.plot_timeseries(2, ts_dpsim_dp_1ph_shift['n2.v_shift'], plt_linestyle='-')
pt.plot_timeseries(3, ts_dpsim_dp_1ph_shift['n3.v_shift'], plt_linestyle='-')
pt.plot_timeseries(4, ts_dpsim_dp_1ph_shift['r1.i_intf_shift'], plt_linestyle='-')

# SP 1Ph shifted
ts_dpsim_sp_1ph_shift['n1.v_shift'].label = 'v1 SP 1Ph'
ts_dpsim_sp_1ph_shift['n2.v_shift'].label = 'v2 SP 1Ph'
ts_dpsim_sp_1ph_shift['n3.v_shift'].label = 'v3 SP 1Ph'
ts_dpsim_sp_1ph_shift['r1.i_intf_shift'].label = 'i12 SP 1Ph'
pt.plot_timeseries(1, ts_dpsim_sp_1ph_shift['n1.v_shift'], plt_linestyle=':')
pt.plot_timeseries(2, ts_dpsim_sp_1ph_shift['n2.v_shift'], plt_linestyle=':')
pt.plot_timeseries(3, ts_dpsim_sp_1ph_shift['n3.v_shift'], plt_linestyle=':')
pt.plot_timeseries(4, ts_dpsim_sp_1ph_shift['r1.i_intf_shift'], plt_linestyle=':')

# Assertions: Compare current through all components

In [ ]:
#1     s ≙ 100'000 steps
#0.025 s ≙ 2500    steps
#0.001 s ≙ 100     steps


#EMT::Ph1 vs EMT::Ph3 assertions

tolerance = 1e-6

#simulation start: Check between 0.025s and 0.03s
assert(np.all(abs(ts_dpsim_emt_1ph['r1.i_intf'].values[2500:3000] - ts_dpsim_emt_3ph['r1.i_intf_0'].values[2500:3000]) < tolerance))
#before open switch: Check between 0.47 and 0.495s
assert(np.all(abs(ts_dpsim_emt_1ph['r1.i_intf'].values[47000:49500] - ts_dpsim_emt_3ph['r1.i_intf_0'].values[47000:49500]) < tolerance))
#after open switch: Check between 0.525s and 0.53s
assert(np.all(abs(ts_dpsim_emt_1ph['r1.i_intf'].values[52500:53000] - ts_dpsim_emt_3ph['r1.i_intf_0'].values[52500:53000]) < tolerance))
#before close switch: Check between 1.47 and 1.495s
assert(np.all(abs(ts_dpsim_emt_1ph['r1.i_intf'].values[147000:149500] - ts_dpsim_emt_3ph['r1.i_intf_0'].values[147000:149500]) < tolerance))
#after close switch: Check between 1.525s and 1.505s
assert(np.all(abs(ts_dpsim_emt_1ph['r1.i_intf'].values[152500:153000] - ts_dpsim_emt_3ph['r1.i_intf_0'].values[152500:153000]) < tolerance))
#simulation end: Check between 1.995s and 2s
assert(np.all(abs(ts_dpsim_emt_1ph['r1.i_intf'].values[199500:200000] - ts_dpsim_emt_3ph['r1.i_intf_0'].values[199500:200000]) < tolerance))

In [ ]:
#EMT::Ph1 vs DP

tolerance = 1e-5

#simulation start: Check between 0.025s and 0.03s
assert(np.all(abs(ts_dpsim_emt_1ph['r1.i_intf'].values[2500:3000] - ts_dpsim_dp_1ph_shift['r1.i_intf_shift'].values[2500:3000]) < tolerance))
#before open switch: Check between 0.47 and 0.495s
assert(np.all(abs(ts_dpsim_emt_1ph['r1.i_intf'].values[47000:49500] - ts_dpsim_dp_1ph_shift['r1.i_intf_shift'].values[47000:49500]) < tolerance))
#after open switch: Check between 0.525s and 0.53s
assert(np.all(abs(ts_dpsim_emt_1ph['r1.i_intf'].values[52500:53000] - ts_dpsim_dp_1ph_shift['r1.i_intf_shift'].values[52500:53000]) < tolerance))
#before close switch: Check between 1.47 and 1.495s
assert(np.all(abs(ts_dpsim_emt_1ph['r1.i_intf'].values[147000:149500] - ts_dpsim_dp_1ph_shift['r1.i_intf_shift'].values[147000:149500]) < tolerance))
#after close switch: Check between 1.525s and 1.505s
assert(np.all(abs(ts_dpsim_emt_1ph['r1.i_intf'].values[152500:153000] - ts_dpsim_dp_1ph_shift['r1.i_intf_shift'].values[152500:153000]) < tolerance))
#simulation end: Check between 1.995s and 2s
assert(np.all(abs(ts_dpsim_emt_1ph['r1.i_intf'].values[199500:200000] - ts_dpsim_dp_1ph_shift['r1.i_intf_shift'].values[199500:200000]) < tolerance))

In [ ]:
#SP vs DP

tolerance =  0.00125

#simulation start: Check between 0.025s and 0.03s
assert(np.all(abs(ts_dpsim_dp_1ph_shift['r1.i_intf_shift'].values[2500:3000] - ts_dpsim_sp_1ph_shift['r1.i_intf_shift'].values[2500:3000]) < tolerance))
#before open switch: Check between 0.47 and 0.495s
assert(np.all(abs(ts_dpsim_dp_1ph_shift['r1.i_intf_shift'].values[47000:49500] - ts_dpsim_sp_1ph_shift['r1.i_intf_shift'].values[47000:49500]) < tolerance))
#after open switch: Check between 0.525s and 0.53s
assert(np.all(abs(ts_dpsim_dp_1ph_shift['r1.i_intf_shift'].values[52500:53000] - ts_dpsim_sp_1ph_shift['r1.i_intf_shift'].values[52500:53000]) < tolerance))
#before close switch: Check between 1.47 and 1.495s
assert(np.all(abs(ts_dpsim_dp_1ph_shift['r1.i_intf_shift'].values[147000:149500] - ts_dpsim_sp_1ph_shift['r1.i_intf_shift'].values[147000:149500]) < tolerance))
#after close switch: Check between 1.525s and 1.505s
assert(np.all(abs(ts_dpsim_dp_1ph_shift['r1.i_intf_shift'].values[152500:153000] - ts_dpsim_sp_1ph_shift['r1.i_intf_shift'].values[152500:153000]) < tolerance))
#simulation end: Check between 1.995s and 2s
assert(np.all(abs(ts_dpsim_dp_1ph_shift['r1.i_intf_shift'].values[199500:200000] - ts_dpsim_sp_1ph_shift['r1.i_intf_shift'].values[199500:200000]) < tolerance))